# Walmart Recruiting - Store Sales Forecasting
- Case técnico que tem por objetivo encontrar um modelo capaz de realizar a projeção de vendas semanais para diferentes departamentos das lojas Walmart.
- Um ponto importante a ser considerado nas análises são os descontos decorrentes de grandes feriados, que podem impactar diretamente no volume das vendas e isso pode variar nas diferentes lojas e dapartamentos.


# Índice

* [Importando bibliotecas necessárias](#import)
* [Upload dos dados](#upload)
* [Consolidação da base analítica](#dataprep)
* [Análise Exploratória](#ead)
* [Tratamento e seleção de variáveis](#features)
* [Modelagem](#models)
* [Interpretação dos resultados](#results)
* [Escorando a base de teste](#scoring)

# Importando bibliotecas necessárias <a class="anchor" id="import"></a>

In [ ]:
import os
from datetime import datetime
import math
from zipfile import ZipFile

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import shap
import pickle

# Upload dos dados <a class="anchor" id="upload"></a>

- Diretório com os dados disponibilizados

In [ ]:
os.listdir('../input/walmart-recruiting-store-sales-forecasting')

- Descompactando os arquivos

In [ ]:
# train
with ZipFile('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip') as f:
    f.extractall(path='walmart-recruiting-store-sales-forecasting')
    
# test    
with ZipFile('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip') as f:
    f.extractall(path='walmart-recruiting-store-sales-forecasting')

# features    
with ZipFile('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip') as f:
    f.extractall(path='walmart-recruiting-store-sales-forecasting')

# sampleSubmission   
with ZipFile('../input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip') as f:
    f.extractall(path='walmart-recruiting-store-sales-forecasting')

os.listdir('walmart-recruiting-store-sales-forecasting')

- Lendo as bases de dados como dataframes usando a biblioteca pandas

In [ ]:
df_train = pd.read_csv("./walmart-recruiting-store-sales-forecasting/train.csv")
df_test = pd.read_csv("./walmart-recruiting-store-sales-forecasting/test.csv")
df_features = pd.read_csv("./walmart-recruiting-store-sales-forecasting/features.csv")
df_stores = pd.read_csv("../input/walmart-recruiting-store-sales-forecasting/stores.csv")
df_submission = pd.read_csv("./walmart-recruiting-store-sales-forecasting/sampleSubmission.csv")

- Verificando a dimensão de cada dataframe

In [ ]:
print("Dimensão dos dataframes:")
print("train", df_train.shape)
print("test", df_test.shape)
print("features", df_features.shape)
print("stores", df_stores.shape)
print("sampleSubmission", df_submission.shape)

# Consolidação da base analítica <a class="anchor" id="dataprep"></a>

### Visualizando e entendendo os dataframes

### Train
- A base de treino contém os seguintes campos:
    - Store: permite identificar o nº da loja, varia de 1 a 45
    - Dept: permite identificar o nº do departamento, varia de 1 a 99
    - Date: semana avaliada, varia de 05-02-2010 a 01-11-2012
    - Weekly_Sales: volume de vendas semanais - variável resposta
    - IsHoliday: permite identificar se a semana possui um dos feriados considerados no estudo 

In [ ]:
df_train.head()

In [ ]:
sorted(df_train.Dept.unique())

### Test
- A base de teste contém os mesmos campos da base de treino, porém sem a informação da variável resposta e com outro intervalo de tempo, o campo Date vai de 02-11-2012 a 26-07-2013.

In [ ]:
df_test.head()

### Stores
- O dataframe abaixo contém as informações sobre o tipo (Type - A, B ou C) e tamanho de cada loja.

In [ ]:
df_stores.head()

In [ ]:
df_stores.Type.unique()

### Features
- O dataframe das features contém os seguintes campos:
    - Store: permite identificar o nº da loja, varia de 1 a 45
    - Date: semana avaliada, varia de 05-02-2010 a 01-11-2012
    - Temperature: temperatura média na região
    - Fuel_Price: preço do combustível na região
    - MarkDown1 a 5: dados relacionados a descontos promocionais; só estão disponíveis após novembro de 2011 e não estão disponíveis para todas as lojas o tempo todo.
    - CPI: inflação
    - Unemployment: a taxa de desemprego
    - IsHoliday: permite identificar se a semana possui um dos feriados considerados no estudo 
    

In [ ]:
df_features.head()

### SampleSubmission
- O último dataframe é um exemplo de como submeter os resultados no kaggle e contém um Id composto por uma combinação do nº da loja, do departamento e da semana, além da previsão do volume de vendas.

In [ ]:
df_submission.head()

### Trazendo as informações das features e das stores pro dataframe de treino
- É importante consolidar as informações em um único dataframe para facilitar o tratamento dos dados e seguir com as próximas análises.
- Para isso, está sendo usada a função `merge` e estão sendo feitos left joins, de modo que toda a informação do treino é mantida e são acrescentadas as informações das demais bases quando as chaves forem iguais.

In [ ]:
df_dev = df_train.merge(df_features
                         ,on = ['Store','Date','IsHoliday']
                         ,how = 'left').merge(df_stores
                                            ,on = ['Store']
                                            ,how = 'left')

df_dev.head()

- Avaliando a quantidade de missings de cada variável

In [ ]:
df_dev.info()

- As únicas variáveis que apresentaram valores faltantes foram as MarkDown1 a 5, o que era esperado, pois esses descontos promocionais começaram após novembro de 2011 e podem variar entre as lojas. Portanto, é interessante seguir com a análise exploratória dos dados para avaliar a melhor forma de trabalhar com essas variáveis.

# Análise Exploratória <a class="anchor" id="ead"></a>
- Essa etapa é fundamental para o entendimento do evento que será modelado e sua relação com as variáveis explicativas que serão avaliadas (features).
- Análises gráficas são bem vindas, pois facilitam a visualização e interpretação do comportamento observado.

### Análise exploratória da variável resposta
**Weekly Sales**
- Por meio da função `describe` pode-se observar que as vendas semanais são representadas por uma variável quantitativa contínua que varia de -4.988,94 a 693.099,36.

In [ ]:
df_dev.Weekly_Sales.describe()

- Desta forma, tem-se um indicativo que algumas lojas tiveram prejuízo, pois apresentaram vendas negativas, o que mostra a relevância de entender quais fatores influenciam no volume de vendas e fazer uma boa previsão para alavancar os resultados.

### Série histórica das vendas
- Primeiramente, foi calculado o volume médio de vendas por semana. Verificou-se que o nº de semanas do período de estudo foram 143 e que as maiores vendas aconteceram, em média, no feriado do Natal e de Ações de Graças.

In [ ]:
# Calculando o volume médio de vendas por semana
w_sales = df_dev.groupby('Date')['Weekly_Sales'].mean()

# Nº de semanas da série 
print("Nº de semanas da série:", len(w_sales))
print("")

# Identificando as datas em que o volume de vendas foi maior
w_sales.sort_values(ascending=False).head()

- Para melhor visualização desse resultado, foi plotado um gráfico da série histórica do volume médio de vendas semanais.

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(w_sales.index, w_sales.values)

plt.xticks(([0,46,98,142]), fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Semana', fontsize=20, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=20, labelpad=20)

plt.title("Volume médio de vendas por semana", fontsize=20);

- Avaliando a série histórica das vendas, pode-se notar que no fim do ano tem-se os maiores picos de vendas, mostrando que os feriados de Ação de Graças e Natal realmente exercem influência sobre o volume das vendas independente da loja ou do departamento. A seguir, foi analisada a variação do volume de vendas com relação à marcação de feriado.

### Volume de vendas $vs$ feriados
- Ao separar a série de vendas pela marcação das semanas que possuem ou não feriado, tem-se que os feriados aconteceram em aproximadamente 7% da série.

In [ ]:
round(df_dev.groupby('IsHoliday')['Weekly_Sales'].count()/df_dev.shape[0]*100)

- Com o auxílio de um boxplot, é possível avaliar os quartis e comparar a distribuição das vendas nas semanas em que houveram ou não feriados.

In [ ]:
plt.figure()
plt.title ('Volume de vendas vs feriados', fontsize=14)
fig = sns.boxplot(x = 'IsHoliday'
                  ,y = 'Weekly_Sales'
                  ,data = df_dev[['Weekly_Sales','IsHoliday']]
                  ,showfliers = True)
plt.xlabel('Feriado', fontsize=14, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=14, labelpad=20);

- No gráfico feito acima, foi utilizado o argumento `showfliers = True` para que os outliers não fossem removidos. Como a série de dados dos feriados já era menor e considerando que valores extremos de vendas eram esperados nessas datas, seria interessante manter os outliers. Assim, pode-se observar que os valores mais extremos de vendas realmente aconteceram em semanas de feriados. Entretando, para melhor visualização dos demais quartis, foi realizado também o boxplot removendo esses outliers.

In [ ]:
plt.figure()
plt.title ('Volume de vendas vs feriados', fontsize=14)
fig = sns.boxplot(x = 'IsHoliday'
                  ,y = 'Weekly_Sales'
                  ,data = df_dev[['Weekly_Sales','IsHoliday']]
                  ,showfliers = False)
plt.xlabel('Feriado', fontsize=14, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=14, labelpad=20);

- Após remover os outliers (`showfliers = False`), pode-se observar que os demais quartis apresentaram valores próximos, indicando que, em geral, a distribuição de vendas é semelhante independente dos feriados. Essa análise vai ao encontro da primeira, concluindo que nem todo feriado interferiu nas vendas e que, em contrapartida, alguns deles proporcionaram valores exorbitantes de vendas.
- Essa análise comprova a relevância desse estudo, pois a previsão do volume de vendas contribui para que as lojas possam fazer um melhor planejamento de seus estoques em épocas de maior demanda e evitam que grandes compras sejam feitas em épocas de menor demanda.

### Volume de vendas nas lojas e departamentos
- A seguir foram construídos boxplots para avaliar se o volume de vendas variava entre as lojas e entre os departamentos. Em ambos os casos foram removidos os outliers para melhor visualização.

**Store**

In [ ]:
plt.figure(figsize = (20,6))
plt.title ('Volume de vendas nas lojas', fontsize=16)
fig = sns.boxplot(x = 'Store'
                  ,y = 'Weekly_Sales'
                  ,data = df_dev[['Store','Weekly_Sales','IsHoliday']]
                  ,showfliers = False)
plt.xlabel('Loja', fontsize=16, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=16, labelpad=20);

- Ao comparar as 45 lojas consideradas, verifica-se que o volume de vendas varia bastante entre elas, o que justifica a importância de considerar uma amostra representativa das diferentes lojas para uma boa previsão do volume de vendas. Abaixo foi quantificada a porcentagem que cada loja representa da série de dados e pode-se observar que está balanceada.

In [ ]:
# Calculando a proporção que cada loja representa na amostra
round(df_dev.groupby('Store')['Store'].count()/df_dev.shape[0]*100,1)

**Dept**
- Fazendo uma comparação análoga na distribuição das vendas dos departamentos, também pode-se notar uma grande diferença no volume das vendas entre eles.

In [ ]:
plt.figure(figsize = (20,8))
plt.title ('Volume de vendas nos departamentos', fontsize=16)
fig = sns.boxplot(x = 'Dept'
                  ,y = 'Weekly_Sales'
                  ,data = df_dev[['Dept','Weekly_Sales']]
                  ,showfliers = False)
plt.xlabel('Departamento', fontsize=16, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=16, labelpad=20);

- Desta forma, foi calculada a quantidade de observações de cada departamento e pode-se observar que alguns departamentos possuem poucas informações, como o nº 39 e o nº 43, por exemplo. Assim, a previsão por departamento pode ser afetada por esse desbalanceamento.

In [ ]:
# Código para mostrar todas as linhas
pd.set_option('display.max_rows', None)

# Calculando a quantidade de observações de cada departamento
df_dev.groupby('Dept')['Dept'].count()

### Análise exploratória das features

**Type**
- Começando pela variável referente ao tipo da loja, foi feito um boxplot em que pode-se observar que o volume de vendas foi maior na empresa A, depois na B, depois na C, ou seja, essa variável se mostrou importante para a previsão das vendas semanais.

In [ ]:
plt.figure(figsize = (20,8))
plt.title ('Volume de vendas nos departamentos',fontsize=16)
fig = sns.boxplot(x = 'Type'
                  ,y = 'Weekly_Sales'
                  ,data = df_dev[['Type','Weekly_Sales']]
                  ,showfliers = False)
plt.xlabel('Tipo da loja', fontsize=16, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=16, labelpad=20);

- Na sequência, tem-se as variáveis quantitativas contínuas, então serão apresentados gráficos de dispersão, coloridos de acordo com o tipo da loja.

**Size**
- No gráfico de dispersão abaixo tem-se a variável tamanho da loja. Por esse gráfico, pode-se notar que as lojas tipo C são as menores, as B são as intermediárias e as maiores são as tipo A. Além disso, tem-se que as menores lojas possuem menores volumes de vendas semanais e que, conforme aumenta o tamanho da loja, as vendas tendem a aumentar também. Essa variável tem potencial para explicar o evento de interesse.

In [ ]:
# Gráfico que relaciona o tamanho da loja e o volume de vendas semanal
plt.figure(figsize=(16,8))
sns.scatterplot(x=df_dev.Size, y=df_dev.Weekly_Sales, hue=df_dev.Type, s=80);

plt.xticks( fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('Tamanho da loja', fontsize=20, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=20, labelpad=20);

**Temperature**
- Numa análise preliminar, observa-se que os maiores volumes de vendas semanais ocorreram quando a temperatura estava mais amena, nem muito quente, nem muito frio, contudo não foi observado um padrão muito marcante no gráfico de dispersão. 

In [ ]:
# Gráfico que relaciona a temperatura e o volume de vendas semanal
plt.figure(figsize=(16,8))
sns.scatterplot(x=df_dev.Temperature, y=df_dev.Weekly_Sales, hue=df_dev.Type, s=80);

plt.xticks( fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('Temperatura', fontsize=20, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=20, labelpad=20);

**Fuel Price**
- O preço do combustível também não parece ter apresentado um impacto muito significativo no volume das vendas. No gráfico abaixo, verificou-se que os maiores valores de vendas ocorreram entre 2,75 e 3,75.

In [ ]:
# Gráfico que relaciona o preço do combustível e o volume de vendas semanal
plt.figure(figsize=(16,8))
sns.scatterplot(x=df_dev.Fuel_Price, y=df_dev.Weekly_Sales, hue=df_dev.Type, s=80);

plt.xticks( fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('Preço do combustível', fontsize=20, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=20, labelpad=20);

**CPI**
- A inflação aparenta ser inversamente proporcional ao volume de vendas semanais, pois os maiores volumes de vendas ocorreram com os menores valores de inflação, no entanto, foi uma diferença muito sutil no nº das vendas.

In [ ]:
# Gráfico que relaciona a Inflação e o volume de vendas semanal
plt.figure(figsize=(16,8))
sns.scatterplot(x=df_dev.CPI, y=df_dev.Weekly_Sales, hue=df_dev.Type, s=80);

plt.xticks( fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('Inflação', fontsize=20, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=20, labelpad=20);

**MarkDown1**
- A primeira variável de descontos apresentou um comportamento inversamente proporcional ao das vendas, ou seja, as maiores vendas ocorreram quando os valores foram próximos a zero. Entretando, vale ressaltar que também houveram volumes baixos de vendas que aconteceram quando os valores foram próximos de zero.

In [ ]:
# Gráfico que relaciona o preço promocional e o volume de vendas semanal
plt.figure(figsize=(16,8))
sns.scatterplot(x=df_dev.MarkDown1, y=df_dev.Weekly_Sales, hue=df_dev.Type, s=80);

plt.xticks( fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('MarkDown 1', fontsize=20, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=20, labelpad=20);

**MarkDown2**
- A segunda variável de descontos apresentou comportamento similar à da primeira, como pode-se observar no gráfico abaixo.

In [ ]:
# Gráfico que relaciona o preço promocional e o volume de vendas semanal
plt.figure(figsize=(16,8))
sns.scatterplot(x=df_dev.MarkDown2, y=df_dev.Weekly_Sales, hue=df_dev.Type, s=80);

plt.xticks( fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('MarkDown 2', fontsize=20, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=20, labelpad=20);

**MarkDown3**
- Por meio do gráfico de dispersão não foi possível notar um padrão claro exercido pela terceira variável de descontos sobre o volume das vendas semanais.

In [ ]:
# Gráfico que relaciona o preço promocional e o volume de vendas semanal
plt.figure(figsize=(16,8))
sns.scatterplot(x=df_dev.MarkDown3, y=df_dev.Weekly_Sales, hue=df_dev.Type, s=80);

plt.xticks( fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('MarkDown 3', fontsize=20, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=20, labelpad=20);

**MarkDown4**
- A quarta variável de descontos apresentou comportamento similar ao da primeira e da segunda, como pode-se observar no gráfico abaixo.

In [ ]:
# Gráfico que relaciona o preço promocional e o volume de vendas semanal
plt.figure(figsize=(16,8))
sns.scatterplot(x=df_dev.MarkDown4, y=df_dev.Weekly_Sales, hue=df_dev.Type, s=80);

plt.xticks( fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('MarkDown 4', fontsize=20, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=20, labelpad=20);

**MarkDown5**
- A quinta variável de descontos apresentou comportamento similar ao da primeira, da segunda e da quanta, como pode-se observar no gráfico abaixo.

In [ ]:
# Gráfico que relaciona o preço promocional e o volume de vendas semanal
plt.figure(figsize=(16,8))
sns.scatterplot(x=df_dev.MarkDown5, y=df_dev.Weekly_Sales, hue=df_dev.Type, s=80);

plt.xticks( fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('MarkDown 5', fontsize=20, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=20, labelpad=20);

# Tratamento e seleção de variáveis <a class="anchor" id="features"></a>

### Preenchendo os missings
- Para uma imputação adequada dos valores faltantes das variáveis MarkDown seria necessário entender mais a fundo sobre elas, contudo, os insights da análise exploratória mostraram que utilizar a mediana é uma boa alternativa. Como os maiores valores de vendas aconteceram, em sua maioria, com preços promocionais próximos a zero e também houveram vendas semanais com valores baixos, quando os descontos estavam próximos a zero, utilizar a mediana é uma opção mais neutra.
- Abaixo foi observada a distribuição de cada variável MarkDown.

In [ ]:
# olhando a distribuição de cada variável MarkDown
df_dev[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].describe()

- Foi utilizada uma função para preenchimento dos missings.

In [ ]:
# Função para preenchimento dos missings

def fillna_MarkDown (df):
    
    '''
    Preenche os missings das variáveis Markdown1 a 5
    Argumento:
        df - dataframe que contém as variáveis
    '''
    
    for i in df.columns:
        if i.startswith('MarkDown'):
            df[i].fillna(df[i].median(), inplace=True)

In [ ]:
fillna_MarkDown(df_dev)
df_dev.head()

- Conferindo se todas as variáveis estão preenchidas.

In [ ]:
df_dev.info()

- Identificando as variáveis como target, numéricas, categóricas e ids. Como a loja e o departamento foram definidas como parte do id na SampleSubmission, foi optado por não utilizar essas variáveis como explicativas no modelo.

In [ ]:
df_dev.columns

In [ ]:
target = ['Weekly_Sales']

numeric_vars = ['Temperature','Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 
                'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size']

categorical_vars = ['IsHoliday', 'Type']

ids = ['Store', 'Dept', 'Date']


### Correlação entre as variáveis numéricas
- Nessa etapa foi avaliada a correlação entre as variáveis explicativas numéricas duas a duas de modo a visualizar a relação entre elas.
- Pode-se notar que as correlações foram fracas entre a grande maioria das variáveis, com exceção das variáveis MarkDown1 e MarkDown4, que apresentaram uma alta correlação positiva (0,82).

In [ ]:
plt.figure(figsize=(16, 16))

correlation = df_dev[numeric_vars].corr(method = 'pearson')
mask = np.triu(np.ones_like(correlation, dtype = bool))

sns.heatmap(correlation, mask = mask, annot = True);

### Transformando as variáveis categóricas em dummies
- Para avaliar a inclusão das variáveis categóricas no modelo, primeiro estas foram transformadas em dummies.

In [ ]:
df_dev_dum = pd.get_dummies(df_dev, columns = categorical_vars)
df_dev_dum.head()

In [ ]:
df_dev_dum.columns

- Após criar as dummies, é importante desconsiderar uma das categorias para evitar multicolinearidade. Isso foi feito eliminando a categoria 'False' da variável 'IsHoliday' e eliminando a categoria 'A' da variável 'Type'.
- Abaixo foi criada uma lista com o nome de todas as variáveis e das variáveis a serem testadas no modelo.

In [ ]:
dummies = ['IsHoliday_True', 'Type_B', 'Type_C']

total_vars = numeric_vars + dummies + ids
model_vars = numeric_vars + dummies

### Dividindo o dataset em treino, teste e validação
- Foi compartilhado um arquivo de treino e um de teste, porém, como o arquivo de teste não possui a variável target e será utilizado apenas para avaliar o resultado final posteriormente, é interessante dividir o arquivo de treino em: desenvolvimento, teste (out of sample - oos) e validação (out of time) para que se possa avaliar os resultados encontrados com o modelo em amostras diferentes da treinada pelo algoritmo.
- Para isso, o campo de data foi transformado em data e, na sequência, quebrado pelo ano e mês de referência.

In [ ]:
df_dev_dum['dt_ref'] = pd.to_datetime(df_dev_dum['Date'])
df_dev_dum['period_month'] = df_dev_dum.dt_ref.dt.to_period('M')

In [ ]:
df_dev_dum.period_month.unique()

- Existem 33 meses no período de estudo, portanto, foram separados os 3 últimos meses para serem usados na validação (out of time), de modo que um dos feriados estivesse incluído nesse período (Labor Day).
- Os primeiros 30 meses foram divididos em treino e teste (out of sample) de modo que 30% foi separado para o teste.

In [ ]:
df_valid = df_dev_dum[(df_dev_dum.period_month == '2012-08') | (df_dev_dum.period_month == '2012-09') | 
         (df_dev_dum.period_month == '2012-10')]
df_dev_split = df_dev_dum[~((df_dev_dum.period_month == '2012-08') | (df_dev_dum.period_month == '2012-09') | 
         (df_dev_dum.period_month == '2012-10'))]

In [ ]:
print(df_valid.shape)
print(df_dev_split.shape)

In [ ]:
# Dividindo a base de treino em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df_dev_split[total_vars], # Variáveis Explicativas + Ids
                                                    df_dev_split[target],  # Variável Resposta
                                                    test_size = 0.3, # Proporção entre treino e teste
                                                    random_state = 0)

In [ ]:
print(X_train.shape)
print(X_test.shape)

### Stepwise
- Como forma de selecionar um conjunto de variáveis que juntas expliquem a variável target de forma significativa, foi utilizada uma função que realiza o stepwise.

In [ ]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [ ]:
stepwise_result = stepwise_selection(X_train[model_vars], y_train)

print('resulting features:')
print(stepwise_result)

- Das variáveis selecionadas, só foram descartadas as MarkDown2 e MarkDown4. Na análise exploratória, as variáveis de descontos em geral se mostraram muito parecidas, além disso, a variável MarkDown4 estava altamente correlacionada à variável MarkDown1, o que indica que apenas uma delas é suficiente para explicar o evento de interesse.
- Abaixo, segue a seleção de variáveis que serão utilizadas na etapa de modelagem.

In [ ]:
vars_stepwise = ['Type_B', 'Type_C', 'Size', 'MarkDown1', 'MarkDown3', 'MarkDown5', 
                 'CPI', 'Unemployment', 'Temperature', 'Fuel_Price', 'IsHoliday_True']

# Modelagem <a class="anchor" id="models"></a>
- Nessa etapa foram testados 3 algoritmos clássicos de machine learning de aprendizagem supervisionada e seu desempenho foi comparado de acordo com a métrica proposta na competição: Weighted Mean Absolute Error (WMAE).

### Regressão Linear
- O primeiro modelo avaliado foi o de Regressão Linear. Esse modelo foi escolhido por ser um modelo simples e de fácil interpretação, assim, pode ser considerado como ponto de partida para as métricas avaliadas.

In [ ]:
X_train = sm.add_constant(X_train) # adding a constant

lr = sm.OLS(y_train, X_train[vars_stepwise]).fit()

# Prediction
y_pred_oos_lr = lr.predict(X_test[vars_stepwise]) 
y_pred_oot_lr = lr.predict(df_valid[vars_stepwise]) 

print_result = lr.summary()
print(print_result)

- Conforme esperado, após realização do stepwise, todas as variáveis foram significativas.
- Em seguida, foi calculado o WMAE com o auxílio da função abaixo.

In [ ]:
def WMAE(model_name, df, target, predictions):
    
    '''
    Calcula a métrica weighted mean absolute error (WMAE) do modelo e salva o resultado em um dataframe.
    Argumentos:
        model_nome - nome do modelo
        df - dataframe que contém a variável IsHoliday_True
        target - variável resposta
        predictions - valores preditos pelo modelo
    '''
    
    weights = df.IsHoliday_True.apply(lambda x: 5 if x==1 else 1)
    
    wmae = np.round(np.sum(weights*abs(target-predictions))/(np.sum(weights)), 2)
    return pd.DataFrame({'Model Name' : model_name,
                        'WMAE' : wmae}, index = [0])

In [ ]:
# DataFrame para salvar as métricas dos modelos
models_metrics = pd.DataFrame()

# WMAE OOS
lr_oos_result = WMAE('Linear Regression OOS', X_test, y_test.Weekly_Sales, y_pred_oos_lr)
lr_oos_result

In [ ]:
# WMAE OOT
lr_oot_result = WMAE('Linear Regression OOT', df_valid, df_valid.Weekly_Sales, y_pred_oot_lr)
lr_oot_result

- Foi calculado o WMAE tanto para a amostra OOS quanto para a OOT e observou-se valores bem parecidos por volta de 14800.

In [ ]:
models_metrics = pd.concat([models_metrics, lr_oos_result],axis = 0)
models_metrics = pd.concat([models_metrics, lr_oot_result],axis = 0)
models_metrics

### Random Forest
- O segundo modelo avaliado foi o Random Forest, que se baseia em uma coleção de árvores de decisão.

In [ ]:
rf = RandomForestRegressor(random_state = 0)

rf.fit(X_train[vars_stepwise], y_train)

# Prediction
y_pred_oos_rf = rf.predict(X_test[vars_stepwise])
y_pred_oot_rf = rf.predict(df_valid[vars_stepwise])

In [ ]:
# WMAE OOS
rf_oos_result = WMAE('Random Forest Regressor OOS', X_test, y_test.Weekly_Sales, y_pred_oos_rf)
rf_oos_result

In [ ]:
# WMAE OOT
rf_oot_result = WMAE('Random Forest Regressor OOT', df_valid, df_valid.Weekly_Sales, y_pred_oot_rf)
rf_oot_result

- Para esse algoritmo, é interessante otimizar os hiperparâmetros do modelo. 
- Devido à limitação de tempo de execução, foram testadas apenas algumas combinações de hiperparâmetros. 

In [ ]:
parameters = {'n_estimators': [100, 200]
             ,'max_depth': [None, 3, 5]
             ,'max_features': [0.25, 0.75]}

rf = RandomForestRegressor()
rf_reg = GridSearchCV(rf, parameters)
rf_reg.fit(X_train[vars_stepwise], y_train)
rf_reg.best_params_

- Com os novos parâmetros definidos, o modelo foi rodado novamente e o WMAE foi realculado.

In [ ]:
rf = RandomForestRegressor(n_estimators = 200, 
                           max_depth = 5,
                           max_features = 0.75,
                           random_state = 0)

rf.fit(X_train[vars_stepwise], y_train)

# Prediction
y_pred_oos_rf = rf.predict(X_test[vars_stepwise])
y_pred_oot_rf = rf.predict(df_valid[vars_stepwise])

In [ ]:
# WMAE OOS
rf_oos_result = WMAE('Random Forest Regressor OOS', X_test, y_test.Weekly_Sales, y_pred_oos_rf)
rf_oos_result

In [ ]:
# WMAE OOT
rf_oot_result = WMAE('Random Forest Regressor OOT', df_valid, df_valid.Weekly_Sales, y_pred_oot_rf)
rf_oot_result

- Após a otimização dos hiperparâmetros, mesmo com uma otimização superficial, pode-se observar uma melhora na métrica WMAE.
- Com uma otimização mais elaborada, a performance do modelo poderia ser ainda melhor.

In [ ]:
models_metrics = pd.concat([models_metrics, rf_oos_result],axis = 0)
models_metrics = pd.concat([models_metrics, rf_oot_result],axis = 0)
models_metrics

- Comparando o WMAE do Random Forest com o da Regressão Linear, pode-se verificar uma melhora, pois os valores diminuíram um pouco, contudo, para uma melhor performance do algoritmo seria importante explorar mais a otimização de parâmetros.
- Para esse modelo, pode-se avaliar a importância das variáveis, como pode-se observar no gráfico construído abaixo.

In [ ]:
importance_df = pd.DataFrame({
    'feature': X_test[vars_stepwise].columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10,6))
plt.title('Feature Importance')
sns.barplot(data=importance_df.head(10), x='importance', y='feature');

- Nesse gráfico, a variável tamanho da loja se destaca como a mais importante do modelo, seguia pela taxa de desempreço, inflação e pelos tipos da loja (B e C).

### Gradient Boosting Regressor - XGBoost
- Por fim, foi avaliado o modelo de Gradient Boosting Regressor que é um técnica Boosting, incluída dentro do grupo de classificadores Ensemble. 

In [ ]:
xgb = GradientBoostingRegressor(random_state = 0)

xgb.fit(X_train[vars_stepwise], y_train)

# prediction
y_pred_oos_xgb = xgb.predict(X_test[vars_stepwise])
y_pred_oot_xgb = xgb.predict(df_valid[vars_stepwise])

In [ ]:
# WMAE OOS
xgb_oos_result = WMAE('XGBoost Regressor OOS', X_test, y_test.Weekly_Sales, y_pred_oos_xgb)
xgb_oos_result

In [ ]:
# WMAE OOT
xgb_oot_result = WMAE('XGBoost Regressor OOT', df_valid, df_valid.Weekly_Sales, y_pred_oot_xgb)
xgb_oot_result

- Para esse algoritmo, também é interessante otimizar os hiperparâmetros do modelo a fim de obter melhores resultados. 
- Devido à limitação de tempo de execução, mais uma vez foram testadas apenas algumas combinações de hiperparâmetros. 

In [ ]:
parameters = {'n_estimators': [100, 200]
             ,'max_depth': [3, 6]
             ,'learning_rate': [0.0001, 0.1]}

xgb = GradientBoostingRegressor()
xgb_reg = GridSearchCV(xgb, parameters)
xgb_reg.fit(X_train[vars_stepwise], y_train)
xgb_reg.best_params_

In [ ]:
xgb = GradientBoostingRegressor(n_estimators = 200
                                ,learning_rate = 0.1
                                ,max_depth = 3
                                ,random_state = 0)

xgb.fit(X_train[vars_stepwise], y_train)

# prediction
y_pred_oos_xgb = xgb.predict(X_test[vars_stepwise])
y_pred_oot_xgb = xgb.predict(df_valid[vars_stepwise])

In [ ]:
# WMAE OOS
xgb_oos_result = WMAE('XGBoost Regressor OOS', X_test, y_test.Weekly_Sales, y_pred_oos_xgb)
xgb_oos_result

In [ ]:
# WMAE OOT
xgb_oot_result = WMAE('XGBoost Regressor OOT', df_valid, df_valid.Weekly_Sales, y_pred_oot_xgb)
xgb_oot_result

- Após a otimização dos hiperparâmetros, verificou-se uma queda muito sutil nos valores do WMAE.
- Com mais tempo e mais capacidade de processamento, mais parâmetros e mais opções poderiam ser exploradas.

### Comparando os três modelos
- Ao avaliar os resultados dos 3 modelos construídos, tem-se que o modelo XGBoost Regressor apresentou valores de WMAE um pouco menores e, portanto, foi o modelo escolhido nesse case.

In [ ]:
models_metrics = pd.concat([models_metrics, xgb_oos_result],axis = 0)
models_metrics = pd.concat([models_metrics, xgb_oot_result],axis = 0)
models_metrics

### Salvando um pickle
- Escolhido o modelo, foi salvo um pickle deste para que a base de test e qualquer outra base nova possa ser escorada.

In [ ]:
XGB_model = pickle.dumps(xgb)

# Interpretação dos resultados <a class="anchor" id="results"></a>
- Em algoritmos de machine leraning, a interpretação das variáveis não é tão direta quanto nos modelos de regressão. Dessa forma, foi feita uso do SHAP (SHapley Additive exPlanations) para auxiliar na compreensão de como o modelo XGBoost Regressor está utilizando as variáveis para fazer as predições.

In [ ]:
# explain the model's predictions using SHAP values
explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(X_train[vars_stepwise])

# summarize the effects of all the features
shap.summary_plot(shap_values, X_train[vars_stepwise])

- Em ordem de importância, as 5 principais variáveis foram interpretadas:

**1. Size:** o tamanho da loja foi a principal variável; quanto maior, maior o nº de vendas

**2. CPI:** a inflação foi a segunda variável mais relevante; quanto menor, maior o nº de vendas

**3. Unemployment:** a taxa de desemprego foi a terceira variável mais relevante; quanto menor, maior o nº de vendas

**4. MarkDown3:** descontos 3 foi a quarta variável mais relevante; quanto maior, maior o nº de vendas

**5. Fuel_Price:** do preço do combustível foi a quinta variável mais relevante; quanto menor, maior o nº de vendas

### Escorando toda a base de treino com o modelo escolhido
- Para comparar a série de vendas histórica observada (target) com a série predita pelo modelo, foi escorada toda a base de treino com o modelo selecionado.

In [ ]:
df_dev_dum['Weekly_Sales_pred'] = xgb.predict(df_dev_dum[vars_stepwise])

In [ ]:
# Calculando o volume médio de vendas por semana
w_sales_pred = df_dev_dum.groupby('Date')['Weekly_Sales_pred'].mean()

plt.figure(figsize=(20,6))
plt.plot(w_sales.index, w_sales.values)
plt.plot(w_sales_pred.index, w_sales_pred.values)

plt.xticks(([0,46,98,142]), fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Semana', fontsize=20, labelpad=20)
plt.ylabel('Volume de vendas', fontsize=20, labelpad=20)

plt.title("Volume médio de vendas por semana", fontsize=20);
plt.legend(['Observado', 'Predito'], fontsize=18);

- No gráfico plotado, pode-se notar que os valores preditos estão próximos dos valores observados, contudo não conseguem prever as oscilações das vendas com precisão.
- Avaliando os 2 grandes picos causados pelos feriados de Natal e Ações de Graças, o modelo consegue prever bem o segundo pico de vendas semanais, mas subestima o primeiro.

# Escorando a base de teste <a class="anchor" id="scoring"></a>
- Como última etapa desse case, foi construído um pipeline para rodar o modelo e, portanto, escorar a base **test**.
- Foram incluídos os códigos necessários para carregar as bibliotecas e funções utilizadas.
- Outro ponto importante foi adaptar a função utilizada para preencher os missings, de modo que permita completar qualquer variável numérica com a mediana e não somente as variáveis MarkDown, dado que, em produção, valores faltantes podem acontecer também nas demais variáveis.
- Também foi criada a coluna do **Id**, necessária para identificar a loja, departamento e semana das vendas.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor
import pickle

# Trazendo todas as informações
df_merge = df_test.merge(df_features
                       ,on = ['Store','Date','IsHoliday']
                       ,how = 'left').merge(df_stores
                                            ,on = ['Store']
                                            ,how = 'left')

# Definindo as variáveis
numeric_vars = ['Temperature','Fuel_Price', 'MarkDown1', 'MarkDown3', 
                'MarkDown5', 'CPI', 'Unemployment', 'Size']

categorical_vars = ['IsHoliday', 'Type']

# Criando o Id
df_merge['Id'] = df_merge['Store'].astype(str) + '_' + df_merge['Dept'].astype(str) + '_' + df_merge['Date']

# Função para preencher os missings
def fillna_numeric (df):    
    '''
    Preenche os missings das variáveis numéricas com a mediana destas
    Argumento:
        df - dataframe que contém as variáveis
    '''   
    for i in numeric_vars:
        df[i].fillna(df[i].median(), inplace=True)

# Preenchendo os missings            
fillna_numeric(df_merge)

# Criando as dummies
df_dum = pd.get_dummies(df_merge, columns = categorical_vars)

dummies = ['IsHoliday_True', 'Type_B', 'Type_C']

# Variáveis do modelo
model_vars = numeric_vars + dummies

# Carregando o pickle do modelo
xgb_model = pickle.loads(XGB_model) 

# Escorando a base teste
df_dum['Weekly_Sales'] = xgb_model.predict(df_dum[model_vars])


In [ ]:
df_test_submission = df_dum[['Id','Weekly_Sales']]
df_test_submission.head()

### Avaliando a série estimada de vendas da base de teste
- Visualizando graficamente os resultados obtidos ao escorar a base test, tem-se um padrão diferente do período de treino, em que os feriados do fim do ano não aparecem evidentes, mas sim o Labor Day, com um pico em fevereiro de 2013.

In [ ]:
# Calculando o volume médio de vendas por semana
w_sales_test_pred = df_dum.groupby('Date')['Weekly_Sales'].mean()

# Nº de semanas da série 
print("Nº de semanas da série:", len(w_sales_test_pred))
print("")

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(w_sales_test_pred.index, w_sales_test_pred.values)

plt.xticks(([0,15,38]), fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Semana', fontsize=16, labelpad=16)
plt.ylabel('Volume de vendas', fontsize=16, labelpad=16)

plt.title("Volume médio de vendas por semana", fontsize=16);